In [1]:
!python3 --version

Python 3.10.6


In [2]:
!pip install langchain==0.0.152
!pip3 install slack_sdk==3.21.0
!pip3 install arxiv==1.4.4
!pip install wikipedia
!pip install einops
!pip install flash-attn==1.0.3.post0 triton==2.0.0.dev20221202

  Using cached flash_attn-1.0.3.post0.tar.gz (2.0 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [21 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-75ln7mu4/flash-attn_0dfb49ad5087444c99c64eef76f1e3f7/setup.py", line 109, in <module>
          _, bare_metal_version = get_cuda_bare_metal_version(CUDA_HOME)
        File "/tmp/pip-install-75ln7mu4/flash-attn_0dfb49ad5087444c99c64eef76f1e3f7/setup.py", line 24, in get_cuda_bare_metal_version
          raw_output = subprocess.check_output([cuda_dir + "/bin/nvcc", "-V"], universal_newlines=True)
        File "/usr/lib/python3.10/subprocess.py", line 420, in check_output
          return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
        File "/usr/lib/python3.10/sub

In [3]:
import gc
gc.collect()

257

In [4]:
from contextlib import contextmanager
from time import time

class Timer:
    """処理時間を表示するクラス
    with Timer(prefix=f'pred cv={i}'):
        y_pred_i = predict(model, loader=test_loader)
    
    with Timer(prefix='fit fold={} '.format(i)):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)

    with Timer(prefix='fit fold={} '.format(i), verbose=500):
        clf.fit(x_train, y_train, 
                eval_set=[(x_valid, y_valid)],  
                early_stopping_rounds=100,
                verbose=verbose)
    """
    def __init__(self, logger=None, format_str='{:.3f}[s]', prefix=None, suffix=None, sep=' ', verbose=0):

        if prefix: format_str = str(prefix) + sep + format_str
        if suffix: format_str = format_str + sep + str(suffix)
        self.format_str = format_str
        self.logger = logger
        self.start = None
        self.end = None
        self.verbose = verbose

    @property
    def duration(self):
        if self.end is None:
            return 0
        return self.end - self.start

    def __enter__(self):
        self.start = time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time()
        out_str = self.format_str.format(self.duration)
        if self.logger:
            self.logger.info(out_str)
        else:
            print(out_str)

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

In [6]:
%%time
# from langchain.llms import OpenAI

# # LLMの準備
# llm = OpenAI(temperature=0.9)

import torch
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, pipeline

device_num = -1 # CPU
if torch.cuda.is_available():
    device_num = 0 # cuda:0

with Timer(prefix=f'set model'):
    tokenizer = AutoTokenizer.from_pretrained(
        "mosaicml/mpt-7b-chat"
    )
    model = AutoModelForCausalLM.from_pretrained(
        "mosaicml/mpt-7b-chat", 
        torch_dtype=torch.float16,
        trust_remote_code=True
    ).to("cuda:0")

    pipe = pipeline(
        "text-generation", model=model, tokenizer=tokenizer,
      max_new_tokens=256, device=device_num, torch_dtype=torch.float16
    )
    # LLMの準備
    llm = HuggingFacePipeline(pipeline=pipe)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-chat/fa9345560420e2111498f87799ea7dd31cbd647e/attention.py:148: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'MPTForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBertForCa

set model 15214.194[s]
CPU times: user 6min 56s, sys: 1min 46s, total: 8min 42s
Wall time: 4h 13min 35s


In [8]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

### SequentialChain

In [9]:
from langchain import PromptTemplate, LLMChain
from langchain.chains import SequentialChain

# 1つ目のチェーンの準備 : タイトルからあらすじを生成
# template = """Please summarize the entire given paper in 128 characters or less, and then output the title, original text and its Japanese translation, summary, novelty or difference, unique methods, and experimental results (if there are numerical evaluation results, please write them in this experimental results) in Japanese in the following format.
# ---
# Output Example 1
# Today, 2023-04-11, is the first paper in the AIOps
# Towards a consistent interpretation of AIOps models
# ## Towards a consistent interpretation of AIOps models
# ## In a nutshell.
# Consistency is needed in the interpretation of AIOps models. In this study, we investigate three aspects of consistency in the interpretation of AIOps models: internal consistency, external consistency, and temporal consistency, for two tasks: failure prediction of Google cluster jobs and failure prediction of Backblaze hard drives. Randomness, hyperparameter adjustment, and data sampling control were found to affect interpretation consistency; AIOps models with AUCs greater than 0.75 provide more consistent interpretations than lower performing models. We also found that AIOps models built with a sliding window or full history approach provide the most consistent interpretation with trends across the data set.
# ### Overview.
# AIOps has been employed by organizations for a variety of tasks, including interpretation of models to identify indicators of service failure. However, many AIOps studies violate established practices in the machine learning community when deriving model interpretations. This study investigates the consistency of AIOps model interpretations and provides guidelines for improving consistency.
# ### Novelty/Differences
# The novelty of this study is that it investigates the consistency of interpretation of AIOps models from three aspects: internal consistency, external consistency, and temporal consistency, and provides guidelines to improve the consistency of interpretation.
# ### Methodology
# We investigated the consistency of interpretation of the AIOps model for two tasks: failure prediction of Google cluster jobs and failure prediction of Backblaze hard drives. We investigated three aspects: internal consistency, external consistency, and temporal consistency, and showed that randomness, hyperparameter adjustment, and data sampling control affect interpretation consistency. We also found that AIOps models with AUCs greater than 0.75 provide more consistent interpretations than lower-performing models. Finally, we found that AIOps models built with a sliding window or full history approach provided the most consistent interpretation of trends across the data set.
# ### Results.
# We found that factors affecting the consistency of interpretation of AIOps models include randomness, hyperparameter adjustment, and data sampling control; we found that AIOps models with AUCs greater than 0.75 provide more consistent interpretation than lower performing models. We also found that AIOps models built with a sliding window or full history approach provided the most consistent interpretation of trends across the data set.
# ### Comments.
# This study investigates three aspects of interpretation consistency in AIOps models: internal consistency, external consistency, and temporal consistency, and provides guidelines for improving interpretation consistency. This will help practitioners avoid misunderstandings in interpreting the AIOps model.
# ---
# Response Format
# text:{text}
# ---
# # Japanese translation of title
# ## In a word.
# ### Summary
# ### Novelty/Difference
# ### Methodology
# ### Result
# ### Comments
# ```"""

template = """与えられた論文の全体を128文字以内にまとめた後、タイトルと原文とその日本語訳、概要、新規性や差分、独特の手法、実験結果（評価結果の数値がある場合は、この実験結果にその内容を書いてください）を以下のフォーマットで日本語で出力してください。```
---
Output Example 1
Decentralized Federated Learning Preserves Model and Data Privacy
分散型連合学習がモデルとデータのプライバシーを保証する
リンク: http://arxiv.org/abs/2102.00880v1
日付: 2021-02-01 14:38:54
ITシステムの機能不全時にサポートするソリューションを求める需要が増えています。
概要
AIOpsは、学術界と産業界において、ますます重視される研究分野ですが、十分なラベル付けされたデータにアクセスすることが困難な問題があります。この問題を解決する方法として、フェデレーテッド・ラーニングが提供されます。この方法では、トレーニングデータへの直接アクセスが必要ないというメリットがあります。しかし、この方法には、中央インスタンスを介してモデルのパラメータを同期させる必要があります。この中央インスタンスが信頼できる必要があり、また、単一障害点になる可能性もあります。そこで、本研究では、フェデレーテッド・ラーニングのフルに分散型アプローチを提案します。このアプローチでは、トレーニング済みモデル間での知識共有が可能です。トレーニングデータやモデルパラメータの送信は必要ありません。代わりに、教師と生徒の役割がモデルに割り当てられ、生徒は教師の出力を元にトレーニングされます。本研究では、ログ異常検知に関する事例研究を実施しま
一言でいうと
分散システムにおいて、異常検知を行うために利用できる多源データを利用する方法を提案しています。
概要
分散システムから生成される多源データを統合的に解析することで、システムの状態の異常を検知することができます。本研究では、分散システムのトレースデータとログデータを統合することで異常検知を行う手法を提案しています。実験により、単一モデリティに基づいた異常検知方法よりも、トレースデータとログデータの統合に基づいた方法の方がより良い結果を生み出すことが示されています。
新規性・差分
以前の研究では、単一モデリティに基づいた異常検知方法が提案されていましたが、本研究ではトレースデータとログデータの統合に基づいた異常検知方法を提案しています。
手法
本研究では、トレースデータとログデータを統合することで異常検知を行う手法を提案しています。
結果
実験により、単一モデリティに基づいた異常検知方法よりも、トレースデータとログデータの統合に基づいた方法の方がより良い結果を生み出すことが示されています。
コメント
分散システムにおいて、多源データを統合的に利用することで、より効果的な異常検知が行えることが示されています。
---
Response Format
text:{text}
---
# タイトルの日本語訳
## 一言でいうと
---
### 概要
### 新規性・差分
### 手法
### 結果
### コメント
```"""

# プロンプトテンプレートの生成
prompt_template = PromptTemplate(
    input_variables=["text"], 
    template=template
)

# LLMChainの準備
result_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    output_key="result"
)

In [10]:
# 2つ目のチェーン : あらすじの発展生成
template = """Describe the following sentence in three words.{result}
```
Word 1:
Word 2:
Word 3:
```"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["result"], 
    template=template
)

# LLMChainの準備
word_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="word"
)

In [11]:
# # 5つ目のチェーン : 道具を英訳する
# template = """あなたは翻訳家です。与えられた文章を日本語に訳してください```
# 文章:{result}
# 翻訳結果:```"""

# # プロンプトテンプレートの準備
# prompt_template = PromptTemplate(
#     input_variables=["result"], 
#     template=template
# )

# # LLMChainの準備
# translation_chain = LLMChain(
#     llm=llm, prompt=prompt_template, 
#     output_key="translation"
# )

In [12]:
# SequentialChainで2つのチェーンを繋ぐ
overall_chain = SequentialChain(
    chains=[result_chain, word_chain],
    input_variables=["text"],
    output_variables=["result", "word"],
    verbose=True
)

# SlackとopenaiのGPTで論文の要約をする

## Reference
- [最新の論文をChatGPTで要約して毎朝Slackに共有してくれるbotを作る！](https://zenn.dev/ozushi/articles/ebe3f47bf50a86)
- [Slack API を使用してメッセージを投稿する](https://zenn.dev/kou_pg_0131/articles/slack-api-post-message)
- [【Slack】インストールするボットユーザーがありませんと出たときの対処方法](https://the-simple.jp/slack-nobotuser#Step1Bot)

In [13]:
import random
import time

from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
import arxiv

In [14]:
# Slack APIトークン

SLACK_API_TOKEN = 'SLACK_API_TOKEN'    # ユーザーとして API を実行するためのトークン 
# Slackに投稿するチャンネル名を指定する
SLACK_CHANNEL = "SLACK_CHANNEL"

In [15]:
# Slack APIクライアントを初期化する
client = WebClient(token=SLACK_API_TOKEN)
#queryを用意
# query_list = ['ti:%22 Anomaly Detection %22', 'ti:%22 AIOps %22']
query_list = ['Anomaly Detection', 'AIOps']
message_list = ['Anomaly Detection', 'AIOps']

# query_list = ['AIOps']
# message_list = ['AIOps']

In [16]:
# from transformers import MarianMTModel, MarianTokenizer
# from transformers import pipeline

In [17]:
# !pip install sentencepiece pysbd

In [18]:
# Load the MarianMTModel and MarianTokenizer for English to Japanese translation
# fugu_translator = pipeline('translation', model='staka/fugumt-en-ja')

In [19]:
for j in range(len(query_list)):
    query = query_list[j]
    # arxiv APIで最新の論文情報を取得する
    search = arxiv.Search(
        query=query,  # 検索クエリ（
        max_results=50,  # 取得する論文数
        sort_by=arxiv.SortCriterion.SubmittedDate,  # 論文を投稿された日付でソートする
        sort_order=arxiv.SortOrder.Descending,  # 新しい論文から順に取得する
    )
    
    #searchの結果をリストに格納
    result_list = []
    for result in search.results():
        result_list.append(result)

    #ランダムにnum_papersの数だけ選ぶ
    num_papers = 5
    results = random.sample(result_list, k=num_papers)
    
    today = time.strftime('%Y-%m-%d', time.localtime())
    # 論文情報をSlackに投稿する
    for i, result in enumerate(results):
        gc.collect()
        print(result)
        try:
            # SequentialChainの実行
            # Slackに投稿するメッセージを組み立てる
            message = overall_chain({"text":result.summary})
            # 翻訳
            # j_text = fugu_translator(message["result"])
            # print(j_text)
            # print(j_text["translation_text"])
            # Slackにメッセージを投稿する
            response = client.chat_postMessage(
                channel=SLACK_CHANNEL,
                # text="本日 " + str(today) + f"{message_list[j]} の" + "論文\n" + f"title: {result.title}\n"+f"link: {result}\n"+str(j_text)
                text="本日 " + str(today) + f"{message_list[j]} の" + "論文\n" + f"title: {result.title}\n"+f"link: {result}\n"+message["result"]
            )
            print(f"Message posted: {response['ts']}")
        except SlackApiError as e:
            print(f"Error posting message: {e}")
            continue

http://arxiv.org/abs/2305.02727v1


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683430135.157659
http://arxiv.org/abs/2305.02490v1


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683430331.113999
http://arxiv.org/abs/2305.03024v1


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683430505.260289
http://arxiv.org/abs/2305.02775v1


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683430685.819559
http://arxiv.org/abs/2305.02559v1


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683430858.039989
http://arxiv.org/abs/2208.03938v1


> Entering new SequentialChain chain...


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.
Message posted: 1683431026.902229
http://arxiv.org/abs/2105.04547v2


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683431195.838739
http://arxiv.org/abs/2209.09645v1


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683431359.390479
http://arxiv.org/abs/2206.15033v2


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683431533.530109
http://arxiv.org/abs/2104.15052v2


> Entering new SequentialChain chain...

> Finished chain.
Message posted: 1683431700.302179


In [20]:
print(message["result"])


Output Example 2
A Novel Deep Learning Framework for Anomaly Detection in Distributed Systems
分散システムの異常検知に対するニューラルネットワークのフレームワークの提案
リンク: http://arxiv.org/abs/2103.01005v1
日付: 2021-03-01 14:36:01
ITシステムの機能不全時にサポートするソリューションを求める需要が増えています。
概要
分散システムの異常検知に対するニューラルネットワークのフレームワークを提案します。
このフレームワークは、異常検知のためのニューラルネットワークを提供し、分散システムの異常検知に対する高い精度を提供します。
このフレー�


In [21]:
print(message["word"])


Output:
A Novel Deep Learning Framework for Anomaly Detection in Distributed Systems
分散システムの異常検知に対するニューラルネットワークのフレームワークの提案
リンク: http://arxiv.org/abs/2103.01005v1
日付: 2021-03-01 14:36:01
ITシステムの機能不全時にサポートするソリューションを求める需要が増えています。
概要
分散システムの異常検知に対するニューラルネットワークのフレームワークを提案します。
このフレームワークは、異常検知のためのニューラルネットワークを提供し、分散システムの異常検知に対する高い精度を提供します。
このフレーズ


## ReAct

In [22]:
from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
# 数学的処理を実行する必要があるときに利用するツールの構築の下準備
# from langchain import LLMMathChain
# llm_math_chain = LLMMathChain(llm=llm, verbose=True)

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    ),
    # # 数学に関する質問に答える必要があるときに利用するツール
    # Tool(
    #     name="Calculator",
    #     func=llm_math_chain.run,
    #     description="useful for when you need to answer questions about math"
    # ),
]

In [23]:
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [29]:
# trans_review = message["word"].split('\n')[1]
trans_review = message["word"].split('\n')[-1]

trans_review

'このフレーズ'

In [25]:
message["result"].split('\n')[1]

'Output Example 2'

In [30]:
trans_review = message["result"]

In [32]:
del llm; del message; gc.collect()

601

In [ ]:
react_res = react.run(trans_review)
# react_res = react.run("What is " + message["result"].split('\n')[1] + "?")

# while react_res == 'Agent stopped due to iteration limit or time limit.':
#     print('retry')
#     react_res = react.run(question)



> Entering new AgentExecutor chain...


In [ ]:
react_res